In [7]:
#engine
#session
#Base

In [8]:
from sqlalchemy.orm import DeclarativeBase

In [ ]:
# Base - A registry which has all ORM modules

class Base(DeclarativeBase):
    """ Base class for all ORM models.

    This class acts as the declarative base for SQLAlchemy ORM models.
    All entity classes (tables) in the application should inherit from this
    class to ensure consistent metadata and mapping behavior

    """

    pass

In [10]:
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy import String

In [ ]:
class Book(Base):

    """Represents a book record in the library database.

    This class defines the ORM mapping for the `Books` table
    Each instance corresponds to a single book entry with its
    unique ID, Title, ISBN, and Department

    Attributes:
        ID (int): Unique identifier for the book (primary key)
        Title (str): Title of the book
        ISBN (str): International Standard Book Number
        Department (str): Department or category the book belongs to
    """

    # Table name in the database
    __tablename__ = "Books"

    # Define the columns in the table using Mapped (Python type) and mapped_column (DB-specific arguments) 

    ID: Mapped[int] = mapped_column(primary_key = True)
    Title: Mapped[str] = mapped_column(String(255))
    ISBN: Mapped[str] = mapped_column(String(20))
    Department: Mapped[str] = mapped_column(String(50))

In [12]:
# Create Engine - created connections to databases

from sqlalchemy import create_engine


In [13]:
# Using an in-memory database for a quick example, or a file for persistence.
# For a file: SQLALCHEMY_DATABASE_URL = "sqlite:///example_20.db"

SQLALCHEMY_DATABASE_URL = "sqlite:///sample.db" 
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, 
    echo=True, # Logs all SQL statements to the console
    connect_args={"check_same_thread": False} # Required for multithreading with SQLite
)

In [14]:
# --- 4. Create Tables using Base.metadata.create_all() ---
if __name__ == "__main__":
    print("Initializing database...")

    # This command inspects all models inherited from Base (in this case, 'User')
    # and generates the DDL (CREATE TABLE statements) to create them in the database.
    # It will not recreate tables that already exist.
    
    Base.metadata.create_all(engine)

Initializing database...
2025-10-13 17:20:16,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-13 17:20:16,120 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Books")
2025-10-13 17:20:16,121 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-13 17:20:16,123 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Books")
2025-10-13 17:20:16,125 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-13 17:20:16,128 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Books" (
	"ID" INTEGER NOT NULL, 
	"Title" VARCHAR(255) NOT NULL, 
	"ISBN" VARCHAR(20) NOT NULL, 
	"Department" VARCHAR(50) NOT NULL, 
	PRIMARY KEY ("ID")
)


2025-10-13 17:20:16,129 INFO sqlalchemy.engine.Engine [no key 0.00091s] ()
2025-10-13 17:20:16,139 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
from sqlalchemy.orm import sessionmaker

In [18]:
# sessionmaker() returns a class; calling it creates a Session instance
SessionLocal = sessionmaker(bind=engine)

In [19]:
# Create a new session
session = SessionLocal()

# Create new Book records
book1 = Book(Title="Deep Learning", ISBN="12-3456789012", Department="AI")
book2 = Book(Title="Data Science Handbook", ISBN="98-7654321098", Department="DS")

# Add records to the session
session.add_all([book1, book2])

# Commit the changes (writes to DB)
session.commit()

# Close the session
session.close()


2025-10-13 17:23:14,135 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-13 17:23:14,140 INFO sqlalchemy.engine.Engine INSERT INTO "Books" ("Title", "ISBN", "Department") VALUES (?, ?, ?) RETURNING "ID"
2025-10-13 17:23:14,142 INFO sqlalchemy.engine.Engine [generated in 0.00024s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Deep Learning', '12-3456789012', 'AI')
2025-10-13 17:23:14,144 INFO sqlalchemy.engine.Engine INSERT INTO "Books" ("Title", "ISBN", "Department") VALUES (?, ?, ?) RETURNING "ID"
2025-10-13 17:23:14,146 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Data Science Handbook', '98-7654321098', 'DS')
2025-10-13 17:23:14,149 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
session = SessionLocal()

# Fetch all books
books = session.query(Book).all()

for book in books:
    print(f"{book.ID} | {book.Title} | {book.ISBN} | {book.Department}")

session.close()


2025-10-13 17:23:44,679 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-13 17:23:44,682 INFO sqlalchemy.engine.Engine SELECT "Books"."ID" AS "Books_ID", "Books"."Title" AS "Books_Title", "Books"."ISBN" AS "Books_ISBN", "Books"."Department" AS "Books_Department" 
FROM "Books"
2025-10-13 17:23:44,683 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ()
1 | Deep Learning | 12-3456789012 | AI
2 | Data Science Handbook | 98-7654321098 | DS
2025-10-13 17:23:44,686 INFO sqlalchemy.engine.Engine ROLLBACK
